In [ ]:
from flask import Flask
import json
from flask import request
from flask import make_response
import pandas as pd
import numpy as np
import datetime
import requests
from google.cloud import datastore
from google.auth._default import _load_credentials_from_file
from google.oauth2 import service_account
from pandas.io import gbq

app = Flask(__name__)

@app.route('/',methods = ['POST'])
def webhook():
    req = request.get_json()
    df_action = req['queryResult']['action'] # To understand which product customer is applying

    i = pd.np.where('credit' in df_action,8,pd.np.where(('broadband' in df_action),3,2))
    
    product = str(req['queryResult']['outputContexts'][i]['parameters']['product.original'])
    
    email = str(req['queryResult']['outputContexts'][i]['parameters']['email'])
    phone_number = str(req['queryResult']['outputContexts'][i]['parameters']['phone-number'])
    dob = req['queryResult']['outputContexts'][i]['parameters']['dateofbirth']
    fb_send_id = req['queryResult']['outputContexts'][i]['parameters']['facebook_sender_id']
    name = str(req['queryResult']['outputContexts'][i]['parameters']['user_fullname'])
    
    current_timestamp = str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
    
    # This is done to push the right campaign to JAPP
    campaign_name = str(pd.np.where("Personal Loan" in product,'5bc3f4ccabf048cb39b9bafc',
                           pd.np.where("Credit Card" in product,'5bc3f454abf048b139b9bafe','5bc3f53cabf048b239b9bafc')))
    
    #url = 'http://apply.imoney.sg/api/save/submission?callback=callback'
    
    #url = 'http://chatbot.im.my/chatbot' ##This was link with Amazon Dyanmo
    #headers = requests.utils.default_headers()
    
    # Payload to push to JAPP
    #payload = {'merge_vars[name]':name,
    #           'merge_vars[email_address]':email,
    #           'merge_vars[phone_number]':phone_number,
    #           'merge_vars[campaign_id]':campaign_name,
    #           'merge_vars[utm_source]':'facebook',
    #           'merge_vars[utm_medium]':'dialogflow_chatbot'
    #          }
    
    # POST data request
    #r = requests.post(url, data=payload,headers=headers)
    
    
    # Upload data to Google DataStore
    #creden = service_account.Credentials.from_service_account_file("https://storage.cloud.google.com/staging.singapore-ea8de.appspot.com/Singapore-3dd7affc7e18.json")
    client = datastore.Client(project='singapore-ea8de')#,credentials=creden)
    
    # Upload to Sg_dialogflow 
    key = client.key('sg_dialogflow')
    entity = datastore.Entity(key=key)
    entity.update({
        'email':str(email),
        'phone_number': str(phone_number),
        'customer_name':str(name),
        'dataofbirth':str(dob),
        'fb_send_id':str(fb_send_id),
        'product_type':str(product),    
        'created_at':str(current_timestamp),
        'japp_status_code':'not_passed'})
    client.put(entity)
    
if __name__ == '__main__':
    app.run(debug=True)